<a href="https://colab.research.google.com/github/JacopoMangiavacchi/FastLabeling/blob/master/mnist_odd_even_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install 'fastai2>=0.0.11'

In [2]:
from fastai2.vision.all import *

matplotlib.rc('image', cmap='Greys')

In [3]:
path = untar_data(URLs.MNIST)
Path.BASE_PATH = path
path.ls()

(#6) [Path('oe_training'),Path('30_training'),Path('oe_testing'),Path('training'),Path('testing'),Path('30_testing')]

In [4]:
(path/'training').ls()

(#10) [Path('training/0'),Path('training/1'),Path('training/7'),Path('training/8'),Path('training/6'),Path('training/3'),Path('training/2'),Path('training/9'),Path('training/4'),Path('training/5')]

In [5]:
from shutil import copy2, rmtree

def copy_files(path, source_folder, destination_folder, sub_folders_list = [], random_select = False, max_files_to_copy = 0):
  def copy_folder_path(source_path, destination_path):
    source_files = source_path.ls()
    list_files = list(range(len(source_files)))
    if random_select:
      random.shuffle(list_files)
    count = 0
    for i in list_files:
      copy2(source_files[i], destination_path)
      count += 1
      if max_files_to_copy > 0 and count >= max_files_to_copy:
        break
  
  (path/destination_folder).mkdir(exist_ok=True)
  if len(sub_folders_list) > 0:
    for i in sub_folders_list:
      folder = source_folder + '/' + str(i)
      copy_folder_path(path/folder, path/destination_folder)
  else:
    copy_folder_path(path/source_folder, path/destination_folder)


In [25]:
rmtree(path/'oe_training', ignore_errors=true)
rmtree(path/'oe_testing', ignore_errors=true)

(path/'oe_training').mkdir(exist_ok=True)
(path/'oe_testing').mkdir(exist_ok=True)
path.ls()

(#4) [Path('oe_training'),Path('oe_testing'),Path('training'),Path('testing')]

In [26]:
copy_files(path, 'training', 'oe_training/odd', [0, 2, 4, 6, 8])
copy_files(path, 'training', 'oe_training/even', [1, 3, 5, 7, 9])
copy_files(path, 'testing', 'oe_testing/odd', [0, 2, 4, 6, 8])
copy_files(path, 'testing', 'oe_testing/even', [1, 3, 5, 7, 9])


In [27]:
(path/'oe_training/odd').ls(), (path/'oe_training/even').ls(), (path/'oe_testing/odd').ls(), (path/'oe_testing/even').ls()

((#29492) [Path('oe_training/odd/55152.png'),Path('oe_training/odd/59372.png'),Path('oe_training/odd/17528.png'),Path('oe_training/odd/51986.png'),Path('oe_training/odd/47486.png'),Path('oe_training/odd/40749.png'),Path('oe_training/odd/49398.png'),Path('oe_training/odd/37646.png'),Path('oe_training/odd/37184.png'),Path('oe_training/odd/51565.png')...],
 (#30508) [Path('oe_training/even/59242.png'),Path('oe_training/even/26571.png'),Path('oe_training/even/47314.png'),Path('oe_training/even/3751.png'),Path('oe_training/even/44687.png'),Path('oe_training/even/5668.png'),Path('oe_training/even/31134.png'),Path('oe_training/even/780.png'),Path('oe_training/even/8330.png'),Path('oe_training/even/44534.png')...],
 (#4926) [Path('oe_testing/odd/8330.png'),Path('oe_testing/odd/6596.png'),Path('oe_testing/odd/6547.png'),Path('oe_testing/odd/542.png'),Path('oe_testing/odd/5393.png'),Path('oe_testing/odd/285.png'),Path('oe_testing/odd/8310.png'),Path('oe_testing/odd/156.png'),Path('oe_testing/odd

In [28]:
size_traning = 100
size_testing = 30
size_training_name = str(size_traning) + '_training'
size_testing_name = str(size_testing) + '_testing'

rmtree(path/size_training_name, ignore_errors=true)
rmtree(path/size_testing_name, ignore_errors=true)

(path/size_training_name).mkdir(exist_ok=True)
(path/size_testing_name).mkdir(exist_ok=True)
path.ls()

(#6) [Path('oe_training'),Path('oe_testing'),Path('100_training'),Path('training'),Path('testing'),Path('30_testing')]

In [29]:
for i in range(10):
  copy_files(path, 'training/' + str(i), size_training_name + '/' + str(i), random_select=True, max_files_to_copy=size_traning)
  copy_files(path, 'testing/' + str(i), size_testing_name + '/' + str(i), random_select=True, max_files_to_copy=size_testing)

In [30]:
dls = ImageDataLoaders.from_folder(path, train='oe_training', valid='oe_testing')

dls.train_ds, dls.valid_ds

Could not do one pass in your dataloader, there is something wrong in it


((#60000) [(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1))...],
 (#10000) [(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size

In [12]:
learn = cnn_learner(dls, resnet34, metrics=accuracy)
learn.fine_tune(3)


epoch,train_loss,valid_loss,accuracy,time
0,0.283479,0.203789,0.917500,01:58


epoch,train_loss,valid_loss,accuracy,time
0,0.046851,0.031010,0.990500,02:10
1,0.027016,0.014745,0.995400,02:09
2,0.011342,0.013495,0.995900,02:10


In [13]:
dls2 = ImageDataLoaders.from_folder(path, train='30_training', valid='30_testing')

dls2.train_ds, dls.valid_ds

((#300) [(PILImage mode=RGB size=28x28, TensorCategory(0)),(PILImage mode=RGB size=28x28, TensorCategory(0)),(PILImage mode=RGB size=28x28, TensorCategory(0)),(PILImage mode=RGB size=28x28, TensorCategory(0)),(PILImage mode=RGB size=28x28, TensorCategory(0)),(PILImage mode=RGB size=28x28, TensorCategory(0)),(PILImage mode=RGB size=28x28, TensorCategory(0)),(PILImage mode=RGB size=28x28, TensorCategory(0)),(PILImage mode=RGB size=28x28, TensorCategory(0)),(PILImage mode=RGB size=28x28, TensorCategory(0))...],
 (#10000) [(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=2

In [31]:
learn2 = cnn_learner(dls2, resnet34, metrics=accuracy, custom_head=learn.model)
learn2.fine_tune(20)

# learn2 = Learner(dls2, learn.model, metrics=accuracy) #, loss_func=CrossEntropyLossFlat(), splitter=default_split)
# learn2.fine_tune(20)


RuntimeError: ignored

In [ ]:
test_path = (path/'testing'/'5').ls()[10]
test_image = Image.open(test_path)
test_image

In [ ]:
learn2.predict(test_path)